In [ ]:
import tensorflow as tf

In [ ]:
TRUE_W = 3.0
TRUE_B = 2.0

NUM_EXAMPLES = 1000

x = tf.random.normal(shape=[NUM_EXAMPLES])

noise = tf.random.normal(shape=[NUM_EXAMPLES])

y = TRUE_W * x + TRUE_B + noise

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(x, y, c='b')
plt.show()

In [ ]:
class MyModel(tf.Module):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        self.w = tf.Variable(5.0)
        self.b = tf.Variable(0.0)
    
    def __call__(self, x):
        return self.w * x + self.b

model = MyModel()

print(model.variables)
print(model(tf.constant([3.0])).numpy())

In [ ]:
def loss(target_y, predicted_y):
    return tf.reduce_mean(tf.square(target_y - predicted_y))

In [ ]:
plt.scatter(x, y, c='b')
plt.scatter(x, model(x), c='r')
plt.show()

In [ ]:
def train(model, x, y, learning_rate):
    with tf.GradientTape() as t:
        current_loss = loss(y, model(x))

    dw, db = t.gradient(current_loss, [model.w, model.b])
    model.w.assign_sub(learning_rate * dw)
    model.b.assign_sub(learning_rate * db)

In [ ]:
model = MyModel()

Ws, bs = [], []
epochs = range(10)

def training_loop(model, x, y):
    for epoch in epochs:
        train(model, x, y, learning_rate=0.1)

        Ws.append(model.w.numpy())
        bs.append(model.b.numpy())

        current_loss = loss(y, model(x))

        print("Epoch %2d: W=%1.2f b=%1.2f loss=%2.5f" % (epoch, Ws[-1], bs[-1], current_loss))

In [ ]:
print("Starting: W=%1.2f b=%1.2f loss=%2.5f" % (model.w, model.b, loss(y, model(x))))

training_loop(model, x, y)

plt.plot(epochs, Ws, 'r')
plt.plot(epochs, bs, 'b')

plt.plot([TRUE_W] * len(epochs), 'r--')
plt.plot([TRUE_B] * len(epochs), 'b--')
plt.legend(['W', 'b', 'True W', 'True b'])
plt.show()

In [ ]:
plt.scatter(x, y, c='b')
plt.scatter(x, model(x), c='r')
plt.show()

In [ ]:
class MyModelKeras(tf.keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        self.w = tf.Variable(5.0)
        self.b = tf.Variable(0.0)
    
    def call(self, x):
        return self.w * x + self.b

keras_model = MyModelKeras()

training_loop(keras_model, x, y)
keras_model.save_weights('my_checkpoint')

In [ ]:
keras_model = MyModelKeras()

keras_model.compile(optimizer=tf.keras.optimizers.SGD(0.1), loss=tf.keras.losses.mean_squared_error, run_eagerly=False)

In [ ]:
print(x.shape)

keras_model.fit(x, y, epochs=10, batch_size=1000)